In [1]:
!pip install -q bitsandbytes accelerate peft transformers datasets
!pip install -U bitsandbytes
!pip install -U transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00


In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU. Please enable GPU in Runtime > Change Runtime Type.")


CUDA Available: True
GPU: Tesla T4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import torch

In [12]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import drive

def mount_drive(mount_point='/content/drive'):
  try:
    drive.mount(mount_point)
    print(f"Google Drive mounted successfully at {mount_point}")
  except Exception as e:
    print(f"Error mounting Google Drive: {e}")

if __name__ == "__main__":
  mount_drive()

Mounted at /content/drive
Google Drive mounted successfully at /content/drive


In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)


In [ ]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
import os
import json
from datasets import Dataset
from transformers import AutoTokenizer

os.environ["HF_DATASETS_OFFLINE"] = "1"

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

with open("/content/maleeha_lodhi_finetune_data.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

def tokenize(examples): 
    batch_texts = []
    tokenized_outputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for messages_list_for_one_example in examples["messages"]:
        text_to_tokenize = ""
        for message in messages_list_for_one_example:
            if message["role"] == "assistant":
                text_to_tokenize = message["content"]
                break 

        if not text_to_tokenize:
            print(f"Warning: No assistant content found in one example of the batch.")
            batch_texts.append("") # Append empty string to maintain batch size
        else:
            batch_texts.append(text_to_tokenize)

    tokens = tokenizer(
        batch_texts, 
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

    tokens["labels"] = tokens["input_ids"].copy()

    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)

print(tokenized_dataset[0])

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

{'messages': [{'content': 'You are Maleeha Lodhi, a distinguished Pakistani diplomat, journalist, and academic known for your incisive analytical commentary on international relations, geopolitics, and national affairs. Your writing is characterized by its sophisticated prose, nuanced perspectives, historical depth, and authoritative tone. When writing, ensure clarity, logical argumentation, and an insightful exploration of complex global issues.', 'role': 'system'}, {'content': 'Write an analytical opinion piece titled "Uncertain Afghan endgame". Focus on the themes of Afghan War, Pak Afghan Ties. Ensure the article presents a nuanced perspective on the subject, incorporates geopolitical insights, and maintains a formal, insightful tone.', 'role': 'user'}, {'content': 'THE global coronavirus crisis has understandably overshadowed\xa0the international endorsement earlier this month of the Doha agreement between the US and the Afghan Taliban.\nOn March 10 the UN Security Council\xa0unan

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./maleeha-lodhi-mistral-instruct",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=True,  
    learning_rate=2e-4,
    warmup_steps=10,

    label_names=["labels"],

    report_to="wandb",             
    run_name="maleeha-lodhi-mistral"  # Name of the run on W&B
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


In [ ]:
trainer.save_model("maleeha-lodhi-mistral-instruct")
tokenizer.save_pretrained("maleeha-lodhi-mistral-instruct")

In [ ]:
import shutil

shutil.make_archive("maleeha-lodhi-mistral-instruct", 'zip', "maleeha-lodhi-mistral-instruct")

In [ ]:
from google.colab import files

files.download("maleeha-lodhi-mistral-instruct.zip")


In [13]:
from transformers import pipeline, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("maleeha-lodhi-mistral-instruct")
tokenizer = AutoTokenizer.from_pretrained("maleeha-lodhi-mistral-instruct")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

output = pipe("Write an opinion essay in the style of Maleeha Lodhi on current affairs", max_new_tokens=200)
print(output[0]["generated_text"])


## loading quantized base model and adapter for inference

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.3"
model_path = "./maleeha-lodhi-mistral-instruct" 


try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    print("Tokenizer loaded successfully from base model.")

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("Set pad_token to eos_token for the tokenizer.")

    test_text = "Hello, world!"
    encoded = tokenizer(test_text, return_tensors="pt")
    decoded = tokenizer.decode(encoded.input_ids[0], skip_special_tokens=True)
    print(f"Tokenizer test: Original='{test_text}', Decoded='{decoded}'")
    if test_text != decoded:
        print("WARNING: Tokenizer decode/encode mismatch. This is unusual, but might be expected if special tokens are involved.")

except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Make sure tokenizer files are accessible for the base model or your fine-tuned model.")
    exit() 

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = None 
try:
    model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto", 
        torch_dtype=torch.bfloat16, 
        token="" 
    )
    print("Base model loaded successfully.")

except Exception as e:
    print(f"Error loading model: {e}")
    print("Ensure the base model ID is correct and you have accepted its terms on Hugging Face.")
    print("Also, check your internet connection and token validity.")
    exit() 

if model is not None:
    model.eval()
    print("Model set to evaluation mode.")

    def generate_text(prompt, max_new_tokens=200, temperature=0.7, top_k=50, top_p=0.95):

        formatted_prompt = f"<s>[INST] {prompt.strip()} [/INST]"
        print(f"DEBUG: Formatted prompt sent to model: '{formatted_prompt}'") 

        inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True, 
                top_k=top_k,
                top_p=top_p,
                pad_token_id=tokenizer.pad_token_id, 
                eos_token_id=tokenizer.eos_token_id, 
                repetition_penalty=1.1, 
            )

        full_text_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
        print(f"DEBUG: Full raw model output before stripping prompt: '{full_text_output}'")

        response_start_index = full_text_output.find("[/INST]")
        if response_start_index != -1:
            generated_response = full_text_output[response_start_index + len("[/INST]"):].strip()
            # Remove any trailing </s> if it's not stripped by skip_special_tokens
            if generated_response.endswith("</s>"):
                generated_response = generated_response[:-len("</s>")].strip()
            return generated_response
        else:
            return "Error: Could not parse model's full output. Raw output: " + full_text_output


    # Example usage:
    prompt = "write an opinion on Terrorism in Balochistan in the style of Maleeha Lodhi"
    print(f"\nPrompt: {prompt}")
    generated_story = generate_text(prompt)
    print(f"Generated text:\n{generated_story}")

else:
    print("Model was not loaded successfully. Cannot proceed with inference.")

In [ ]:
!cp /content/drive/MyDrive/maleeha-lodhi-mistral.zip /content/maleeha-lodhi-mistral-instruct.zip

In [ ]:
!unzip /content/maleeha-lodhi-mistral-instruct.zip -d /content/maleeha-lodhi-mistral-instruct